In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle
pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [ ]:
# Read in the data
INPUT_DIR = 'm5-data'
cal = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
stv = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
ss = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
sellp = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')

We are given historic sales data in the `sales_train_validation` dataset.
- rows exist in this dataset for days d_1 to d_1913. We are given the department, category, state, and store id of the item.
- d_1914 - d_1941 represents the `validation` rows which we will predict in stage 1
- d_1942 - d_1969 represents the `evaluation` rows which we will predict for the final competition standings.

In [ ]:
stv.head()

## Merging the data with real dates
- We are given a calendar with additional information about past and future dates.
- The calendar data can be merged with our days data
- From this we can find weekly and annual trends

In [ ]:
# Calendar data looks like this (only showing columns we care about for now)
cal[['d','date','event_name_1','event_name_2',
     'event_type_1','event_type_2', 'snap_CA']].head()

In [ ]:
d_cols = [c for c in stv.columns if 'd_' in c] # sales data columns

# Below we are chaining the following steps in pandas:
# 1. Select the item.
# 2. Set the id as the index, Keep only sales data columns
# 3. Transform so it's a column
# 4. Plot the data
stv.loc[stv['id'] == 'FOODS_3_090_CA_3_validation'] \
    .set_index('id')[d_cols] \
    .T \
    .plot(figsize=(15, 5),
          title='FOODS_3_090_CA_3 sales by "d" number',
          color=next(color_cycle))
plt.legend('')
plt.show()

In [ ]:
# Merge calendar on our items' data
example = stv.loc[stv['id'] == 'FOODS_3_090_CA_3_validation'][d_cols].T
example = example.rename(columns={8412:'FOODS_3_090_CA_3'}) # Name it correctly
example = example.reset_index().rename(columns={'index': 'd'}) # make the index "d"
example = example.merge(cal, how='left', validate='1:1')
example.set_index('date')['FOODS_3_090_CA_3'] \
    .plot(figsize=(15, 5),
          color=next(color_cycle),
          title='FOODS_3_090_CA_3 sales by actual sale dates')
plt.show()

In [ ]:
stv.columns[:6]

In [ ]:
print(f'Number of unique items: {np.unique(stv.item_id).shape[0]}')
print(f'Number of unique department: {np.unique(stv.dept_id).shape[0]}')
print(f'Number of unique categories: {np.unique(stv.cat_id).shape[0]}')
print(f'Number of unique stores: {np.unique(stv.store_id).shape[0]}')
print(f'Number of unique states: {np.unique(stv.state_id).shape[0]}')

In [ ]:
# Transform column wide days to single column 

stv = stv.melt(list(stv.columns[:6]), var_name='day', value_vars=list(stv.columns[6:]), ignore_index=True)

In [ ]:
# Group by the groups to consider (remove product_id as there are 3049 unique) 

stv = stv.groupby(['dept_id', 'cat_id', 'store_id', 'state_id', 'day']).sum('value').reset_index()

In [ ]:
days_calendar = np.concatenate((stv['day'].unique().reshape(-1,1), cal['date'][:-56].unique().reshape(-1,1)), axis=1)
df_caldays = pd.DataFrame(days_calendar, columns = ['day','Date'])

In [ ]:
# Add calendar days

stv = stv.merge(df_caldays, how='left', on='day')

In [ ]:
stv['Date'] = stv['Date'].astype('datetime64[ns]')
stv.dtypes

In [ ]:
# Transform in weekly data

stv_weekly = stv.groupby(['dept_id', 'cat_id', 'store_id', 'state_id']).resample('W', on='Date')['value'].sum()

In [ ]:
stv_weekly

In [ ]:
# Build the structure to then apply the grouping transformation

stv_pivot = stv_weekly.reset_index().pivot(index='Date',columns=['dept_id', 'cat_id', 'store_id', 'state_id'], values='value')
stv_pivot = stv_pivot.fillna(0)

In [ ]:
stv_pivot

In [ ]:
# Get first series

series_1 = stv_pivot.iloc[:,0]

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics import tsaplots as smplots
plt.rc("figure", figsize=(20,15))
plt.rc("font", size=20)

In [ ]:
decomposition = sm.tsa.seasonal_decompose(series_1, model='additive')
fig = decomposition.plot()
plt.show()

# Time Series Decomposition

### Trend

Now let’s begin the step by step decomposition of this time series.

STEP 1: Try to guess the duration of the seasonal component in your data. In the above example, we’ll guess it to be 52 weeks.

STEP 2: Now run a 52 week centered moving average on the data. This moving average is spread across a total of 53 weeks. i.e. 26 weeks each on the left and right side of the center week. The 52 week centered MA is an average of two moving averages that are shifted from each other by 1 week, effectively making it a weighted moving average.

In [ ]:
df1 = series_1.to_frame()

In [ ]:
#Add an empty column to store the 2x12 centered MA values
df1['trend'] = np.nan#Fill it up with the 2x12 centered MA values

In [ ]:
df1.columns = [' '.join(col).strip() for col in df1.columns.values]

In [ ]:
df1.columns = ['sales', 'trend']

In [ ]:
df1_ = df1.copy()
for i in range(26, df1.shape[0]-26):
    total_i = (
        # Sum the values from the first and last week weighted 
        df1_['sales'].iloc[i - 26] * 1.0 / (52*2) + df1_['sales'].iloc[i + 26] * 1.0 / (52*2))
    for j in range(-25, 26):
        # Add the remaining values weighted by 1/52 (they belong to both MA, so they have double of the weight)
        total_i += df1_['sales'].iloc[i+j]/52
    df1['trend'].iloc[i] = np.round(total_i)

In [ ]:
# Confirming first value (2011-07-31)

(sum(df1['sales'][0:52])/52 + sum(df1['sales'][1:53])/52)/2

In [ ]:
plt.plot(df1_t['trend']);

STEP 3: Now we have a decision to make. Depending on whether the composition is multiplicative or additive, we’ll need to divide or subtract the trend component from the original time series to retrieve the seasonal and noise components. If we inspect the original sales time series, we can see that the seasonal swings are not increasing in proportion to the current value of the time series. Hence we’ll assume that the seasonality is additive. We’ll also take a small leap of faith to assume that the noise is additive.

Thus the retail used car sales time series is assumed to have the following multiplicative decomposition model:

Time series value = trend component + seasonal component + noise component

Therefore:

seasonal component + noise component = Time series value - trend component

We’ll add a new column into our data frame and fill it with the sum of the seasonal and noise components using the above formula.

In [ ]:
df1['seasonality&noise'] = df1['sales']-df1['trend']

In [ ]:
plt.plot(df1['seasonality&noise'])

STEP 4: Next, we will get the ‘pure’ seasonal component out of the mixture of seasonality and noise, by calculating the average value of the seasonal component for all first weeks of January, all second weeks of February and so on.

In [ ]:
df1['week'] = df1.index.isocalendar().week

In [ ]:
df1['seasonality'] = np.nan

In [ ]:
df1_ = df1.copy()
for i in df1['week'].unique():
    df1['seasonality'].iloc[i] = np.sum(df1_.loc[df1_['week']==i,'seasonality&noise'])/df1_[df1_['week']==i].shape[0]

In [ ]:
df1_.loc[df1_['week']==1]

In [ ]:
plt.plot(df1['seasonality']);